In [1]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

## Settings and Dataset

In [3]:
## Settings

# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# hyperparameters
random_seed = 123
learning_rate = 0.1
num_epochs = 10
batch_size = 256

# architecture
num_features = 784
num_classes = 70

# MNIST dataset

train_dataset = datasets.MNIST(root='data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='data',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

# checking
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


In [4]:
## Model

class SoftmaxRegression(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, num_classes)

        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zero_()

    def forward(self, x):
        logits = self.linear(x)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

model = SoftmaxRegression(num_features=num_features,
                          num_classes=num_classes)

model.to(device)

# cost and optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
# manual seed for deterministic data loader
torch.manual_seed(random_seed)

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0

    for features, targets in data_loader:
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()

    return correct_pred.float() / num_examples * 100

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):

        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)

        # forward and backprop
        logits, probas = model(features)

        # note pytorch corss entropy takes in logits instead of probas
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        cost.backward()

        optimizer.step()

        # logging
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader))) 

Epoch: 001/010 | Batch 000/234 | Cost: 0.5039
Epoch: 001/010 | Batch 050/234 | Cost: 0.4120
Epoch: 001/010 | Batch 100/234 | Cost: 0.3381
Epoch: 001/010 | Batch 150/234 | Cost: 0.4571
Epoch: 001/010 | Batch 200/234 | Cost: 0.4656
Epoch: 001/010 training accuracy: 89.32%
Epoch: 002/010 | Batch 000/234 | Cost: 0.4025
Epoch: 002/010 | Batch 050/234 | Cost: 0.3127
Epoch: 002/010 | Batch 100/234 | Cost: 0.4108
Epoch: 002/010 | Batch 150/234 | Cost: 0.3294
Epoch: 002/010 | Batch 200/234 | Cost: 0.4515
Epoch: 002/010 training accuracy: 89.86%
Epoch: 003/010 | Batch 000/234 | Cost: 0.3746
Epoch: 003/010 | Batch 050/234 | Cost: 0.4154
Epoch: 003/010 | Batch 100/234 | Cost: 0.3610
Epoch: 003/010 | Batch 150/234 | Cost: 0.3686
Epoch: 003/010 | Batch 200/234 | Cost: 0.2767
Epoch: 003/010 training accuracy: 90.37%
Epoch: 004/010 | Batch 000/234 | Cost: 0.4124
Epoch: 004/010 | Batch 050/234 | Cost: 0.3743
Epoch: 004/010 | Batch 100/234 | Cost: 0.3609
Epoch: 004/010 | Batch 150/234 | Cost: 0.3751
Epo

In [9]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 91.71%
